In [1]:
import pandas as pd
import pymysql
import datetime

In [17]:
conn = pymysql.connect(host='rr-wz9y03t9wtq075179o.mysql.rds.aliyuncs.com', user='medbuser', passwd='Ptmesys!!!1290098', db='mobility_exchange', port = 3306)
sql = 'SELECT TIME(FROM_TIME) as TIME,AVG_RUNNING, TRIPS_COUNT FROM me_transport_hub_minute_stat WHERE HUB_CODE="SZ_AIR_BA" AND DATE(FROM_TIME)="{}" ORDER BY CRT_ON;'

AVG_RUNNING = pd.DataFrame()
TRIPS_COUNT = pd.DataFrame()

for i in range(14):
    date_i = datetime.date.today() - datetime.timedelta(days=(i+1))
    sql_i = sql.format(date_i)
    df_i = pd.read_sql_query(sql_i, con=conn)
    avg_i = pd.DataFrame()
    trips_i = pd.DataFrame()
    if len(AVG_RUNNING.columns)==0:
        AVG_RUNNING[[date_i,"TIME"]] = df_i[["AVG_RUNNING","TIME"]]
        TRIPS_COUNT[[date_i,"TIME"]] = df_i[["TRIPS_COUNT","TIME"]]
    else:
        avg_i[[date_i, "TIME"]] = df_i[["AVG_RUNNING", "TIME"]]
        trips_i[[date_i, "TIME"]] = df_i[["TRIPS_COUNT", "TIME"]]
        AVG_RUNNING = pd.merge(AVG_RUNNING, avg_i[[date_i,"TIME"]], how="outer", on="TIME")
        TRIPS_COUNT = pd.merge(TRIPS_COUNT, avg_i[[date_i,"TIME"]], how="outer", on="TIME")
AVG_RUNNING.drop_duplicates(inplace=True)
TRIPS_COUNT.drop_duplicates(inplace=True)
AVG_RUNNING.reset_index(drop=True,inplace=True)
TRIPS_COUNT.reset_index(drop=True,inplace=True)
del AVG_RUNNING["TIME"]
del TRIPS_COUNT["TIME"]
AVG_RUNNING.to_csv("AVG_RUNNING_{}_created.csv".format(datetime.date.today()))
TRIPS_COUNT.to_csv("TRIPS_COUNT_{}_created.csv".format(datetime.date.today()))